In [1]:
from dotenv import load_dotenv
import os
import langchain 
from langchain.chat_models import init_chat_model
import getpass

In [2]:
load_dotenv()

True

In [3]:
os.environ["OPENAI_API_KEY"]=os.environ.get("OPENAI_KEY")

In [ ]:
if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

In [4]:

model = init_chat_model("gpt-4o-mini", model_provider="openai")

In [5]:
model.invoke("hi how are you?")

AIMessage(content="Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 12, 'total_tokens': 41, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': None, 'id': 'chatcmpl-Brpf4rcguNCwW5gWgrBMqC11KzAYs', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--8e4c0d4f-a0a3-4979-b57d-a312093773ee-0', usage_metadata={'input_tokens': 12, 'output_tokens': 29, 'total_tokens': 41, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [1]:
from typing import Annotated

from typing_extensions import TypedDict,Literal
import random

from langgraph.graph import StateGraph, START,END
from langgraph.graph.message import add_messages

In [5]:
class State(TypedDict):
     messages: Annotated[list, add_messages]
     

In [20]:
graph=StateGraph(State)

In [21]:
def superbot(state:State):
    return {"messages":[model.invoke(state["messages"])]}

#### ADD nodes and Edgs

In [22]:
#Add Nodes
graph.add_node("SuperBot",superbot)
#Add Edges
graph.add_edge(START,"SuperBot")
graph.add_edge("SuperBot",END)

In [23]:
graph_builder=graph.compile()

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph_builder.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [26]:
graph_builder.invoke({'messages':"Hi my name Abhijeet"})

{'messages': [HumanMessage(content='Hi my name Abhijeet', additional_kwargs={}, response_metadata={}, id='85474222-080a-49ae-acba-0c325db3752b'),
  AIMessage(content='Hi Abhijeet! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 14, 'total_tokens': 27, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': None, 'id': 'chatcmpl-BrpvM5lUwqR2lFzWqwM3S4kgucZ3k', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--c1557f29-3d2a-40d9-8259-3a5d3e60f175-0', usage_metadata={'input_tokens': 14, 'output_tokens': 13, 'total_tokens': 27, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}

In [2]:
#### TypeDict

class Typedictstate(TypedDict):
    name:str
    game:Literal["Cricket","Badminton"]

In [29]:
def playgame(state:Typedictstate):
    return {"name":state["name"]+ "want to play"}

def cricket(state:Typedictstate):
    print("cricket game is call off")
    return {"name":state["name"]+ "cricket","game":"cricket"}

def badminton(state:Typedictstate):
    print("badminton game is call off")
    return {"name":state["name"]+ "badminton","game":"badminton"}

def choose_game(state:Typedictstate)->Literal["cricket","badminton"]:
    if random.random()<0.5:
        return "cricket"
    else:
        return "badminton"
        



In [30]:
graph_2=StateGraph(Typedictstate)

In [31]:
# Add Nodes
graph_2.add_node("playgame",playgame)
#graph.add_node("choose_game",choose_game)
graph_2.add_node("cricket",cricket)
graph_2.add_node("badminton",badminton)

In [32]:
#Add Edges
graph_2.add_edge(START,"playgame")
graph_2.add_conditional_edges("playgame",choose_game)
graph_2.add_edge("cricket",END)
graph_2.add_edge("badminton",END)

In [33]:
builder=graph_2.compile()

In [ ]:
from IPython.display import Image, display
display(Image(builder.get_graph().draw_mermaid_png()))

In [34]:
builder.invoke({"name":"Abhijeet"})

cricket game is call off


{'name': 'Abhijeetwant to playcricket', 'game': 'cricket'}

### Using BaseModel (Pydantic)

In [36]:
from pydantic import BaseModel

In [56]:
class State(BaseModel):
    name:str
    #game:Literal["cricket","badminton"]

In [62]:
def playgame(state:State):
    return {"name": state.name }
def cricket(state:State):
    print("cricket game is call off")
    return {"name": state.name, "game": "cricket"}
def badminton(state:State):
    print("badminton game is call off")
    return {"name": state.name , "game": "badminton"}
def choose_game(state:State) -> Literal["cricket", "badminton"]:
    if random.random() < 0.5:
        return "cricket"
    else:
        return "badminton"

In [63]:
graph3=StateGraph(State)

graph3.add_node("playgame",playgame)
graph3.add_node("cricket", cricket)
graph3.add_node("badminton", badminton)

graph3.add_edge(START,"playgame")
graph3.add_conditional_edges("playgame", choose_game)
graph3.add_edge("cricket", END)
graph3.add_edge("badminton", END)

In [64]:
graph_builder=graph3.compile()

In [66]:
graph_builder.invoke({"name":123})

ValidationError: 1 validation error for State
name
  Input should be a valid string [type=string_type, input_value=123, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type